In [1]:
'''
Application 主架構
'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson = json.load(open('./line_secret_key', 'r', encoding='utf-8'))
server_url = secretFileContentJson.get('server_url')
channel_access_token = secretFileContentJson["channel_access_token"]


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")

#生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get('channel_access_token'))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route('/', methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [2]:
'''
消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。
'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_message_array(fileName):
    
    #打開檔案轉成json格式
    with open(fileName, encoding='utf-8') as f:
        jsonArray = json.load(f)
    
    #解析json
    returnArray = []
    for jsonObject in jsonArray:
        
        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray
        
    
    
        

In [3]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)



uriQuickReplyButton = QuickReplyButton(
    action=URIAction(
        label="西甲", 
        uri="line://app/1614188410-mgw5LXGz"
    )
)

textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息", 
        text="line://app/1614188410-mgw5LXGz"
    )
)

In [4]:
quickReplyList = QuickReply(
    items = [textQuickReplyButton]
)

In [5]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
quickReplyTextSendMessage = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [6]:
'''
將選單綁定到特定用戶上

取得rich_menu_id
取得self_user_id
設定line的遠端位置
設定消息的基本安全驗證
發送消息告知

'''
# #取得rich_menu_id
# link_rich_menu_id = json.load(open('./material/rich_menu/rich_menu_main/rich_menu_id.json', 'r', encoding='utf-8'))
# link_rich_menu_id = link_rich_menu_id['richMenuId']

# #取得self_user_id
# self_user_id = json.load(open('./line_secret_key', 'r', encoding='utf-8'))
# self_user_id = self_user_id['self_user_id']

# # 將菜單id與用戶id組合成遠端位置
# linkMenuEndpoint = 'https://api.line.me/v2/bot/user/%s/richmenu/%s' % (self_user_id, link_rich_menu_id)

# # 設定消息基本安全憑證
# linkMenuRequestHeader = {'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# # 發送消息
# lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
# print(lineLinkMenuResponse)
# print(lineLinkMenuResponse.text)

'\n將選單綁定到特定用戶上\n\n取得rich_menu_id\n取得self_user_id\n設定line的遠端位置\n設定消息的基本安全驗證\n發送消息告知\n\n'

In [7]:
# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    result_message_array  = []
    replyJsonPath = './material/follow/reply.json'
    result_message_array = detect_json_array_to_message_array(replyJsonPath)
    
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )

In [8]:
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs

@handler.add(PostbackEvent)
def process_post_back_event(event):

    query_string_list = parse_qs(event.postback.data)
    print(query_string_list)
    print(query_string_list['change_menu'])
    if 'change_menu' in query_string_list:
        
        #將用戶與另一張圖文選單綁定
        Path = './material/rich_menu/rich_menu_' + query_string_list['change_menu'][0]
        linkRichMenuId = json.load(open(Path + '/rich_menu_id.json', 'r', encoding='utf-8'))['richMenuId']
        line_bot_api.link_rich_menu_to_user(event.source.user_id, linkRichMenuId)
        
        #回傳訊息
        replyJsonPath = './material/rich_menu/' + query_string_list['change_menu'][0] + '_reply'
        result_message_array = detect_json_array_to_message_array(replyJsonPath + '/reply.json')
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'quick_reply' in query_string_list:
        replyJsonPath = './material/rich_menu/rich_menu_' + query_string_list['change_menu']
        print(query_string_list)
        print(query_string_list['quick_reply'])
    elif 'shoes_type' in query_string_list:
        replyJsonPath = './material/rich_menu/rich_menu_shoe' + query_string_list['shoes_type']
        print(query_string_list)
        print(query_string_list['shoes_type'])


In [9]:
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# @handler.add(MessageEvent, message=TextMessage)
# def handle_message(event):
#     line_bot_api.reply_message(
#         event.reply_token,
#         quickReplyTextSendMessage
#     )


In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Sep/2019 23:43:31] "POST / HTTP/1.1" 200 -


{'change_menu': ['shoes']}
['shoes']
U27915f51b4978ddee0207fac63f46b9d
./material/rich_menu/rich_menu_shoes
richmenu-bd47038fd3208a25b97f1635c53ab842


127.0.0.1 - - [26/Sep/2019 23:43:32] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Sep/2019 23:43:41] "POST / HTTP/1.1" 200 -
[2019-09-26 23:43:41,910] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\scott\Anaconda3\envs\linechat\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\scott\Anaconda3\envs\linechat\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\scott\Anaconda3\envs\linechat\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\scott\Anaconda3\envs\linechat\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\scott\Anaconda3\envs\linechat\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\scott\Anaconda3\envs\linechat\lib\site-p

{'shoes_type': ['speed']}
